In [8]:
import torch
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
from torchvision.models import resnet18
import os
from tcav import TCAV

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=4)

model = resnet18()
model = model.cuda()
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters())


def train():
    best_weights = model.state_dict()
    best_acc = 0.0
    for epoch in range(1, 201):

        # train phase
        model.train()
        for inputs, targets in trainloader:
            inputs, targets = inputs.cuda(), targets.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            print(outputs.size())
            print(targets.size())
            loss = criterion(outputs, targets)
            loss.backward()
            #print('loss: ',loss.item())
            optimizer.step()
        
        # test phase
        total = 0
        score = 0
        with torch.no_grad():
            model.eval()
            for inputs, targets in testloader:
                inputs, targets = inputs.cuda(), targets.cuda()
                outputs = model(inputs)
                predicted = outputs.max(dim=1)[1]
                total += targets.size(0)
                score += predicted.eq(targets).sum().item()
        acc = score / total
        print("epoch: {}\tacc: {}".format(epoch, acc))
        if acc > best_acc:
            best_acc = acc
            best_weights = model.state_dict()

    # save model parameters
    torch.save(best_weights, 'resnet18_office.pth')


def validate():
    model.eval()
    model.load_state_dict(torch.load('resnet18_office.pth'))
    # TODO: Create DataLoaders for Broden concepts and train TCAV


Files already downloaded and verified
Files already downloaded and verified


In [9]:
train()

torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size

torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size([128])
torch.Size([128, 1000])
torch.Size

KeyboardInterrupt: 

In [3]:
model.load_state_dict(torch.load('resnet18_cifar10.pth'))

<All keys matched successfully>

In [4]:
from model_wrapper import ModelWrapper
wrappped_model = ModelWrapper(model, ['layer4'])

In [5]:
tcav = TCAV(model, input_dataloader, concept_dataloaders, num_classes, max_samples)

NameError: name 'input_dataloader' is not defined